In [ ]:
import os
import torch
import json
from utils import videos_to_frames, extract_faces, interpolate_missing_frames, resize_excess, pad_resize, save_as_tensor
from torch.utils.data import DataLoader

In [ ]:
from model import FakeNet
model = FakeNet()
model = torch.nn.DataParallel(model)
print(model)
#optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [ ]:
load_name = os.path.join('fake_class2_29_checkpoint.pth.tar')
loaded_model = torch.load(load_name, map_location=torch.device('cpu'))
model.load_state_dict(loaded_model['state_dict'])
#optimizer.load_state_dict(loaded_model['optimizer'])
start_epoch = loaded_model['epoch']

### Run following lines to test on full data
If you don't want please skip, datasize is approximately 4GB

In [ ]:
%env KAGGLE_USERNAME=""
%env KAGGLE_KEY=""

In [ ]:
!kaggle competitions download -c deepfake-detection-challenge

In [ ]:
!unzip deepfake-detection-challenge.zip

In [ ]:
with open(os.path.join("deepfake-detection-challenge", "train_sample_videos", "metadata.json"), 'r') as j:
    metadata = j.read()

filedata = metadata.replace('"REAL"', '1')
filedata = filedata.replace('"FAKE"', '0')

with open(os.path.join("deepfake-detection-challenge", "train_sample_videos", "metadata_edited.json"), 'w') as file:
  file.write(filedata)

### Following lines preprocess the data
demo_data_path should be the name of folder which encloses videos

In [ ]:
demo_data_path = "demo_data"
videos_to_frames(demo_data_path)
extract_faces(demo_data_path)
interpolate_missing_frames(demo_data_path)
resize_excess(demo_data_path)
pad_resize(demo_data_path)

### Following line saves images as one tensor

In [ ]:
save_as_tensor(demo_data_path)

In [ ]:
from data_loader import FakeDetectDataset
dataset = FakeDetectDataset(demo_data_path)
data_loader_params = {'dataset': dataset, 'batch_size': 1, 'shuffle': False, 'sampler': None,
                        'batch_sampler': None, 'num_workers': 0, 'collate_fn': None}
eval_loader = DataLoader(**data_loader_params)

In [ ]:
model.eval()
with torch.no_grad():
    for (batch_no, sample) in enumerate(eval_loader):
            inp = sample[0]
            label = torch.tensor(sample[1], dtype=torch.float)
            output = model(inp)
            output = output.squeeze()
            number_to_probs = lambda class_int: "REAL prob: {:.2f} FAKE probability: {:.2f}".format(torch.sigmoid(output), 1-torch.sigmoid(output).item()) 
            number_to_class = lambda class_int: "REAL" if  class_int > 0.5 else "FAKE"
            print(output)
            class_pred = torch.sigmoid(output)

            acc = torch.sum(class_pred == label) / len(label)
            print(f"Prediction = {number_to_probs(class_pred.item())}", end="\t")
            print(f"Actual = {number_to_class(label.item())}")